In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mplstereonet               #stereonet package 
import ternary as te              #python-ternary diagram package
from numpy.linalg import eigh
from matplotlib.backends.backend_pdf import PdfPages #This is to generate a multipage pdf for all the figures

In [ ]:
os.chdir(r'C:\specify_PATH_to_Folder')   #This is choose the folder containing one or more csv files to work with

In [ ]:
pwd  #This is to check the path

In [ ]:
os.listdir()  #List all the files in the chosen folder

In [ ]:
filelist=glob.glob('*.csv') #creates a list of all csv files

In [ ]:
print(filelist)

In [ ]:
value=filelist[0]   #This is to show that a list can be sliced by index
print(value)

In [ ]:
print(value[:-4]) #And this shows that the string of filename can also be sliced. This will be useful for labels on figures and output filenames

## 1.0 Data

In [ ]:
#Select the csv file to work with. Index 0 is the first file in the list

index=int(input("What is the index nb of the file?"))  #This ensures that the index value is treated as an integer (not a string)




In [ ]:
df=[pd.read_csv(file) for file in filelist][index]   #This will read the chosen csv file

In [ ]:
df

In [ ]:
#Create an array from the 'trend' column of the dataframe and display horizontally (one line)
Trend=df.loc[:,'Trend']
Trend=np.hstack(Trend)

In [ ]:
#Do the same for the plunge data

Plunge=df.loc[:,'Plunge']
Plunge=np.hstack(Plunge)

## 2.0 Data projections

In [ ]:
#Let's prepare the data to build a rose diagram

#Calculate the number of directions (bins) every 10° using numpy.histogram.

bin_edges = np.arange(-5, 366, 10)                              #numpy.arange(start, stop, step)
trends_in_bins, bin_edges = np.histogram(Trend, bin_edges)

In [ ]:
Trends=trends_in_bins[0:-1]                  #37 bins

In [ ]:
#Initialize the pdf file that will contain all the figures
pp = PdfPages(f'Figures_{filelist[index][:-4]}.pdf')


#Create the rose diagram and the stereonets.

fig = plt.figure(figsize=(12, 12))         #creates an empty figure with no Axes

ax1 = fig.add_subplot(221, projection='stereonet')
ax1.line(Plunge, Trend, 'o', color='blue')
ax1.set_title(f'a-axis on stereonet from {filelist[index][:-4]}', y=1.10, fontsize=15)

#Rose diagram
ax2 = fig.add_subplot(222, projection='polar')
ax2.set_title('a-axis on rose diagram', y=1.10, fontsize=15)

ax2.bar(np.deg2rad(np.arange(0, 360, 10)), Trends, 
       width=np.deg2rad(10), bottom=0.0, color='.8', edgecolor='k')
ax2.set_theta_zero_location('N')
ax2.set_theta_direction(-1)
ax2.set_thetagrids(np.arange(0, 360, 30), labels=np.arange(0, 360, 30))
ax2.set_rgrids(np.arange(1, Trends.max() + 4, 3), angle=0, weight= 'black')

ax3 = fig.add_subplot(223, projection='stereonet')
ax3.line(Plunge, Trend, 'o', color='blue')
m=ax3.density_contourf(Plunge, Trend, measurement='lines', cmap='Reds') #exponential_kamb is the default method
ax3.set_title('with kamb density contours', y=1.05, fontsize=15)
fig.colorbar(m) 


# Plot these things on the two stereonets
for ax in [ax1, ax3]:
    ax.grid()
    ax.set_azimuth_ticks([]) #This is to hide the azimuth labels bc there is currently a problem with this in mplstereonet

for ax in [ax1]:
    note = f"n={Plunge.size} \nMean Plunge: {Plunge.mean():.1f}"
    ax.annotate(note, xy=(5*60, -30), xycoords='axes points')  

plt.show()
#option to save a figure separately fig.savefig(f'Stereonets_Rose_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(fig)  #This will save the figure in the multipage pdf


## 3.0 Orientation tensors

In [ ]:
#Calculate direction cosines...

a=np.cos(Trend*np.pi/180)
b=np.cos(Plunge*np.pi/180)     
c=np.sin(Trend*np.pi/180)
xi=(a*b).round(4) #first direction cosines.
yi=(c*b).round(4) #second direction cosines
zi=np.sin(Plunge*np.pi/180).round(4) #third direction cosines

In [ ]:
X=np.concatenate((xi,yi,zi)).reshape(3,Trend.shape[0])    #Brings the three direction cosines (xi, yi, zi) together in a single 3XN array/matrix (N=nb# of measurements)

In [ ]:
XT=X.T               #Transpose matrix X
print(XT)

In [ ]:
A=np.dot(X,XT).round(4)    #3X3 matrix of the sums of cross products of the direction cosines
print(A)

In [ ]:
#Get the eigenvalues and eigenvectors using mplstereonet

plu, azi, vals = mplstereonet.eigenvectors(Plunge, Trend, measurement='lines') 
#This returns 1-D arrays for plunge and azimuth (eigenvectors converted to spherical coordinates), and normalized eigenvalues
print(plu,azi,vals)

In [ ]:
#Extract the normalized eigenvalues
S1=vals[0].round(4)
S2=vals[1].round(4)
S3=vals[2].round(4)
print(S1,S2,S3)
#The eigenvalues are slightly different with mplstereonet than with Stereonet 11

In [ ]:
Ei=eigh(A)  #This returns a 1-D array (eigenvalues) and a 2-D array (eigenvectors)

In [ ]:
Eighv=Ei[0] #Extract the eigenvalues
print(Eighv)

In [ ]:
#The sum of eigenvalues equals to the nb# of measurements. 
#Normalized eigenvalues (divided by number of measurements) sum to 1.

S1=Eighv[2]/Trend.shape
S2=Eighv[1]/Trend.shape
S3=Eighv[0]/Trend.shape
print(S1, S2, S3)  
#normalized eigenvalues; the values obtained with eigh are the same as those obtained using Stereonet 11

In [ ]:
#Make a biplot of S3 and S1

fig = plt.figure(figsize=(10,8))
ax = fig.add_subplot(1,1,1) # row-col-num

# Hide the right and top lines of the default box
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

#ax.set_xlim((0.4, 1.0,))
#ax.set_ylim((0.0, 0.3))

xticks = np.arange(0.4, 1.0, 0.1).round(2)
yticks = np.arange(0.0, 0.3, 0.05).round(2)

xtickLocations=np.arange(0.4, 1.0, 0.1).round(2)
yticklocations=np.arange(0.0, 0.3, 0.05).round(2)


ax.scatter(S1,S3, c='green', s=100)
ax.set_xlabel('S1', fontsize=18)
ax.set_ylabel('S3', fontsize=18)
ax.set_xticks(ticks=xtickLocations)
ax.set_yticks(ticks=yticklocations)
ax.set_xticklabels(xticks, fontsize=16)
ax.set_yticklabels(yticks, fontsize=16)

ax.grid(True, linestyle='--')
plt.show()
#fig.savefig(f'S3_S1plot_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(fig)

In [ ]:
#Extract V1, V2, and V3
V1_azi=azi[0]
V1_azi=V1_azi.round(1)
V1_plunge=plu[0]
V1_plunge=V1_plunge.round(1)
V2_azi=azi[1]
V2_azi=V2_azi.round(1)
V2_plunge=plu[1]
V2_plunge=V2_plunge.round(1)
V3_azi=azi[2]
V3_azi=V3_azi.round(1)
V3_plunge=plu[2]
V3_plunge=V3_plunge.round(1)
print("V1 is", V1_plunge, "\u2192", V1_azi)
print("V2 is", V2_plunge, "\u2192", V2_azi)
print("V3 is", V3_plunge, "\u2192", V3_azi)

In [ ]:
E=(1-(S2/S1)).round(4)     #Elongation index

In [ ]:
I=(S3/S1).round(4)        #Isotropy index

In [ ]:
R=(1-(E+I)).round(4)    #A residual value to allow plotting the indices correctly on a ternary diagram

In [ ]:
df=pd.DataFrame(columns=['V1_trend', 'V1_plunge', 'E','I', 'R'])

In [ ]:
df.loc[0]=[V1_azi, V1_plunge, E,I,R]
df

In [ ]:
df["E"]=df['E'].astype('float')      #To specify the Dtype is 'float'
df["I"]=df['I'].astype('float')
df["R"]=df['R'].astype('float')
df.dtypes

In [ ]:
df.info()

In [ ]:
df

In [ ]:
df.to_csv(f'Data_output_{filelist[index][:-4]}.csv', index=False)

In [ ]:
#This will ask the user to classify the fabric modality

Mod=input("What is the modality of the fabric? ")

if Mod=="un" or Mod=="su" or Mod=="bi" or Mod=="sb" or Mod=="mm":
    print("Thank you")
else:
    print("This is not a valid answer")


In [ ]:
if Mod=='un':
    Modal=0
elif Mod=='su':
    Modal=1
elif Mod=="bi":
    Modal=2
elif Mod=="sb":
    Modal=3
elif Mod=="mm":
    Modal=4

In [ ]:
#Create the modality-isotropy plot

fig = plt.figure(figsize=(12.5,8))
ax = fig.add_subplot(1,1,1) # row-col-num

ylocations=[0,1,2,3,4]
labels=["un", 'su', 'bi', 'sb', 'mm']

# Hide the right and top lines of the default box
ax.spines['right'].set_visible(False)
ax.spines['top'].set_visible(False)

#PLot the dots and assign ticks and labels
ax.scatter(I, Modal, s=200)
ax.set_yticks(ticks=ylocations)
ax.set_yticklabels(labels, fontsize=18)

xticks = [ 0, 0.1, 0.2, 0.3, 0.4 ]
ax.set_xticks(xticks)
ax.set_xticklabels(xticks, fontsize=18)

plt.xlabel(r'S3/S1 (isotropy)', fontsize=16)
plt.ylabel('Modality', fontsize=18)

#Add the grid lines and show the plot
ax.grid(True, linestyle='--')
plt.show()
#fig.savefig(f'Modality_IsotropyPlot_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(fig)

In [ ]:
#Let's plot the results on a ternary diagram!

scale = 1.0
figure, fabric = te.figure(scale=scale)
figure.set_size_inches(12,10)

#PLot the data
fabric.scatter(df[['E','I','R']].values, marker='D', color='green', label="Green Diamonds")
    
# Draw Boundary and Gridlines
fabric.boundary(linewidth=2.0)
fabric.gridlines(color="blue", multiple=0.2)
    
# Set Axis labels
fontsize = 12
offset = 0.2
fabric.left_axis_label("I=S3/S1", fontsize=fontsize, offset=0.2)
fabric.right_axis_label("E=1-(S2/S1)", fontsize=fontsize, offset=0.2)
fabric.top_corner_label("Isotropic", fontsize=fontsize, offset=0.25)
fabric.right_corner_label("Cluster", fontsize=fontsize, offset=-0.05)
fabric.left_corner_label("Girdle", fontsize=fontsize, offset=-0.05)

#This is to configure the style of the axes and ticks and specify their orientation/sense
fabric.ticks(axis='lbr', multiple=0.2, linewidth=1, offset=0.025, tick_formats="%.1f", clockwise=True)
fabric.get_axes().axis('off')
fabric.clear_matplotlib_ticks()

fabric.show()
#fabric.savefig(f'Ternary_Diagram_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(figure)

In [ ]:
#Now V1 can be added to the data points on the stereonet

fig = plt.figure(figsize=(8,8))
ax = fig.add_subplot(111, projection='stereonet')
ax.line(Plunge, Trend, 'o', color='blue', label='a-axis')
plunge=V1_plunge
bearing=V1_azi
ax.line(plunge, bearing, 'X', color='green', markersize=12, label="V1")
ax.set_title('a-axis on stereonet with V1', y=1.10, fontsize=15)


#This is to display the legend in the upper right corner without overlapping the stereonet
ax.legend(loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=14)

#This is another approach to remove duplicate labels in the legend
#def legend_without_duplicate_labels(ax):
    #handles, labels = ax.get_legend_handles_labels()
    #unique = [(h, l) for i, (h, l) in enumerate(zip(handles, labels)) if l not in labels[:i]]
    #ax.legend(*zip(*unique))
    
#Another approach to avoid repetition of labels in the legend:
from collections import OrderedDict

handles, labels = ax.get_legend_handles_labels()
by_label = OrderedDict(zip(labels, handles))

#This is to display the legend in the upper right corner without overlapping the stereonet
ax.legend(by_label.values(), by_label.keys(), loc='upper right', bbox_to_anchor=(1.1, 1.1), fontsize=14)


#Add some notes and values beside the plot
note = f"n={Plunge.size} \nS1={S1} \nV1 azimuth={V1_azi}"
ax.annotate(note, xy=(5*60, -30), xycoords='axes points')



ax.grid()
ax.set_azimuth_ticks([])
plt.show()
#fig.savefig(f'Stereonet_w_V1_{filelist[index]}.svg', bbox_inches = 'tight', format='svg')
pp.savefig(fig)

In [ ]:
pp.close()  #This is to close the pdf file

In [ ]:
#Output a proposed interpretation based on the above results and plots

if Mod=='un' or Mod=='su' and S1>=0.7 and I<0.12:
    print('Very strong till clast fabric; Can be used for paleo-ice flow')
elif Mod=='bi' or Mod=='sb' and S1>=0.7 and I<0.2:
    print("Strong or moderately strong till clast fabric; use for paleo-ice flow with caution")
elif Mod=='sb' and S1>0.55 and S1<0.7 and I>=0.12:
    print("Moderate till clast fabric; interpret paleo-ice flow with great caution; check other data and local context")
elif S1<=0.55 and I>=0.12:
    print("Weak till clast fabric; unreliable for paleo-ice flow")
else: print("Undefined; check data and plots")


In [ ]:
#This will save the interpretation output in a *.txt file

with open(f"Interpretation_{filelist[index][:-4]}.txt", "w") as external_file:
    if Mod=='un' or Mod=='su' and S1>=0.7 and I<0.12:
        print('Very strong till clast fabric; Can be used for paleo-ice flow', file=external_file)
    elif Mod=='bi' or Mod=='sb' and S1>=0.7 and I<0.2:
        print("Strong or moderately strong till clast fabric; use for paleo-ice flow with caution", file=external_file)
    elif Mod=='sb' and S1>0.55 and S1<0.7 and I>=0.12:
        print("Moderate till clast fabric; interpret paleo-ice flow with great caution; check other data and local context", file=external_file)
    elif S1<=0.55 and I>=0.12:
        print("Weak till clast fabric; unreliable for paleo-ice flow", file=external_file)
    else: print("Undefined; check data and plots")
external_file.close()

In [ ]:
if V1_azi>=0 and V1_azi<180:
    Ice_Flow=(V1_azi+180).round()
else: 
    Ice_Flow=(V1_azi-180).round()
          

In [ ]:
#Provide a paleo-ice flow direction based on V1 (if S1>0.57)

if S1>0.57 and V1_plunge>2:
    print('Paleo-ice flow was likely toward', Ice_Flow)
elif S1>0.57 and V1_plunge<2:
    print("Orientation likely reliable but V1 plunge too low; direction could be opposite; check local context")
else: 
    print("Paleo-ice flow direction is uncertain/unreliable")

In [ ]:
#This will add the ice flow interpretation to the same *.txt file as above, but on a different line

with open(f"Interpretation_{filelist[index][:-4]}.txt", "a+") as external_file:
    external_file.seek(0)          #This will go to the top/start of text
    data = external_file.read()
    if len(data) > 0 :
        external_file.write("\n")
    if S1>0.57 and V1_plunge>2:
        external_file.write(f'Paleo-ice flow was likely toward {Ice_Flow}')
    elif S1>0.57 and V1_plunge<2:
        external_file.write("Orientation likely reliable but V1 plunge too low; direction could be opposite; check local context")
    else: 
        external_file.write("Paleo-ice flow direction is uncertain/unreliable")